In [1]:
import pandas as pd
import numpy as np

X_df = pd.read_csv('../03_input/combined_running_data.csv')

In [2]:
X_df['datetime'] = pd.to_datetime(X_df['date'])
# shift the data by one day
X_df['datetime'] = X_df['datetime'] - pd.DateOffset(days=1)
X_df['datetime']


0       2022-07-12
1       2022-07-12
2       2022-07-12
3       2022-07-12
4       2022-07-12
           ...    
60352   2022-07-21
60353   2022-07-21
60354   2022-07-21
60355   2022-07-21
60356   2022-07-21
Name: datetime, Length: 60357, dtype: datetime64[ns]

In [3]:
feature_df = pd.read_csv('../03_input/feature.csv')
feature_df['datetime'] = pd.to_datetime(feature_df['datetime_date'])
feature_df

,datetime_date,month_7,month_8,week_of_month_1,week_of_month_2,week_of_month_3,week_of_month_4,week_of_month_5,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,Wind Speed_int,Temperature_int,Humidity_int,datetime
0,2022-07-31,True,False,False,False,False,False,True,False,False,False,False,False,False,True,4,84,79,2022-07-31
1,2022-07-30,True,False,False,False,False,False,True,False,False,False,False,False,True,False,7,82,79,2022-07-30
2,2022-07-29,True,False,False,False,False,False,True,False,False,False,False,True,False,False,2,86,58,2022-07-29
3,2022-07-28,True,False,False,False,False,True,False,False,False,False,True,False,False,False,9,73,88,2022-07-28
4,2022-07-27,True,False,False,False,False,True,False,False,False,True,False,False,False,False,4,77,89,2022-07-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2022-08-05,False,True,True,False,False,False,False,False,False,False,False,True,False,False,2,90,66,2022-08-05
58,2022-08-04,False,True,True,False,False,False,False,False,False,False,True,False,False,False,4,90,75,2022-08-04
59,2022-08-03,False,True,True,False,False,False,False,False,False,True,False,False,False,False,7,90,75,2022-08-03
60,2022-08-02,False,True,True,False,False,False,False,False,True,False,False,False,False,False,2,88,75,2022-08-02


In [4]:
grid_height = 31
grid_width = 33

grouped_by_date = X_df.groupby("date")

reshaped_data = []

for date, group in grouped_by_date:

    reshaped_grid = group["keep_num"].to_numpy().reshape(grid_height, grid_width, -1)
    # flattened_grid = reshaped_grid.flatten()  # 展平为一维数组

    # 将数据添加到列表中
    reshaped_data.append((date, reshaped_grid))

    # reshaped_data.append(group["keep_length"].to_numpy().reshape(grid_height, grid_width))

In [5]:
reshaped_array = np.array([x[1] for x in reshaped_data])
reshaped_array.shape

(59, 31, 33, 1)

In [6]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers

# # Load and preprocess data
# X_df = pd.read_csv('../03_input/combined_running_data.csv')
# additional_data = pd.read_csv('../03_input/additional_features.csv')  # Load your additional features data

# # Ensure that the date columns match and are properly formatted
# additional_data['date'] = pd.to_datetime(additional_data['date'])
# X_df['date'] = pd.to_datetime(X_df['date'])
# data_combined = pd.merge(X_df, additional_data, on='date')

# # Normalize or standardize your additional features here if necessary
# # This step is critical especially for features like Temperature, Wind Speed, and Humidity
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# # One-hot features don't need scaling
# scale_cols = ['Wind Speed_int', 'Temperature_int', 'Humidity_int']
# additional_data[scale_cols] = scaler.fit_transform(additional_data[scale_cols])

# # Group and reshape data by date
# grouped_by_date = data_combined.groupby("date")
# reshaped_data = []
# for date, group in grouped_by_date:
#     reshaped_grid = group["keep_num"].values.reshape(31, 33, 1)  # Reshape and keep spatial dimensions
#     additional_features = group.drop(['date', 'keep_num'], axis=1).values  # Extract additional features
#     reshaped_data.append((date, reshaped_grid, additional_features))

# # Prepare sequential input
# sequence_length = 6
# input_data, target_values = [], []
# for i in range(len(reshaped_data) - sequence_length):
#     sequence = [reshaped_data[j][1:] for j in range(i, i + sequence_length)]
#     grids = np.array([s[0] for s in sequence])
#     features = np.array([s[1] for s in sequence])
#     input_data.append(np.concatenate([grids, features[:, None, None, :]], axis=-1))  # Merge features into grid
#     target_values.append(reshaped_data[i + sequence_length][1][0])  # Next grid

# input_data = np.array(input_data)
# target_values = np.array(target_values)

# # Define the model
# model = keras.Sequential([
#     layers.TimeDistributed(layers.Conv2D(16, (3, 3), activation='relu'), input_shape=(6, 31, 33, 1 + additional_features.shape[1])),
#     layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
#     layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu')),
#     layers.TimeDistributed(layers.Flatten()),
#     layers.LSTM(50, return_sequences=True),
#     layers.Dense(31*33, activation='linear')  # Adjust according to your output requirements
# ])

# model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# # Train the model
# model.fit(input_data, target_values, epochs=300, batch_size=8, validation_split=0.2)

# # Evaluate the model
# model.evaluate(input_data, target_values)


# modeling

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


## target value set up
这里我们有两种可能的set up,

第一个是对于明天的预测

第二个事对于一系列的未来的预测

我们先从简单的入手. 进行-1的shift, 我们可以得到明天的预测

In [8]:
import numpy as np

# 输入数据集
input_data = []
# 目标数据集
target_values = []

# 这里我们创建了基于时间步的输入和目标数据集
# 使用循环来遍历数据，确保输入的长度是6，并且目标值是接下来的1个时间步

for i in range(len(reshaped_array) - 7):
    # 输入数据是连续的6个时间步
    input_data.append(reshaped_array[i:i+7, :])

    # 目标值是紧接在这6个时间步后的那个时间步
    target_values.append(reshaped_array[i+7, :])

# 将输入数据和目标值转换为numpy数组
input_data = np.array(input_data)
target_values = np.array(target_values)


In [9]:
reshaped_array.shape, input_data.shape, target_values.shape

((59, 31, 33, 1), (52, 7, 31, 33, 1), (52, 31, 33, 1))

In [10]:
# change the shape of the target_values from (n, 31, 33, 1) to (n, 31*33)
target_values = target_values.reshape(-1, 31*33)

In [11]:
target_values.shape

(52, 1023)

In [12]:
from tensorflow.keras import layers, models

# 模型结构
model = models.Sequential([
    layers.TimeDistributed(layers.Conv2D(16, (3, 3), activation='relu'), input_shape=(7, 31, 33, 1)),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu')),
    layers.TimeDistributed(layers.Flatten()),  # 将二维特征展平
    layers.LSTM(50, activation='relu', return_sequences=False),  # 确保 `return_sequences=False`
    layers.Dense(31*33, activation='linear')  # 输出层，应匹配目标值的形状
])

# # Define the model 
# model = keras.Sequential([ 
#     layers.TimeDistributed(layers.Conv2D(16, (3, 3), activation='relu'), input_shape=(6, 31, 33, 1 + additional_features.shape[1])), 
#     layers.TimeDistributed(layers.MaxPooling2D((2, 2))), 
#     layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu')), 
#     layers.TimeDistributed(layers.Flatten()), 
#     layers.LSTM(50, return_sequences=True), 
#     layers.Dense(31*33, activation='linear') # Adjust according to your output requirements 
#     ])

# 编译模型
model.compile(
    optimizer='adam',
    loss='mse',  # 使用均方误差
    metrics=['mae']  # 平均绝对误差
)

# 训练模型
model.fit(
    input_data,  # 输入数据
    target_values,  # 目标值
    epochs=1000,  # 训练轮数
    batch_size=8,  # 批量大小
    validation_split=0.3  # 20% 用于验证
)


Epoch 1/1000


/opt/anaconda3/envs/6c51/lib/python3.12/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 93.9969 - mae: 6.7007 - val_loss: 161.6063 - val_mae: 8.3518
Epoch 2/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 80.4715 - mae: 6.0101 - val_loss: 143.4458 - val_mae: 7.2330
Epoch 3/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 74.5911 - mae: 5.4990 - val_loss: 134.5462 - val_mae: 6.9755
Epoch 4/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 64.3376 - mae: 5.0758 - val_loss: 112.5051 - val_mae: 6.6101
Epoch 5/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 53.4721 - mae: 4.8190 - val_loss: 89.7382 - val_mae: 5.6592
Epoch 6/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 38.1593 - mae: 3.9791 - val_loss: 66.2696 - val_mae: 4.8099
Epoch 7/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 26.2976 - mae: 3.2638 - val_loss: 49.5534 - val_mae: 4.0554
Epoch 8/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 17.9061 - mae: 2.5778 - val_loss: 38.5034 - val_mae: 3.5351
Epoch 9/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss

In [13]:
# 在测试集上评估模型
model.evaluate(input_data, target_values)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.2160 - mae: 1.1185 


[8.909167289733887, 1.3646113872528076]

In [14]:
# mae / target_values.mean()
model.predict(input_data).shape

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step


(52, 1023)

In [16]:
print(target_values.shape)
# change the shape of the target_values from (n, 31*33) to (n, 31, 33, 1)
target_values = target_values.reshape(-1, 31, 33, 1)
target_values.shape


(52, 1023)


(52, 31, 33, 1)

In [18]:
# visulizer
reshaped_array.shape

(59, 31, 33, 1)